In [1]:
import datetime
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [2]:
def peek(df):
    display(df.iloc[0:3, :])
    print(len(df))

In [3]:
def repair_dates(df):
    df['date_range_start'] = pd.to_datetime(df['date_range_start'], utc=True)
    df['date_range_start'] = df['date_range_start'].dt.tz_convert('US/Eastern')

In [ ]:
def smoothen(df, columns=[], N=6):
    """Returns a copy of the given dataframe with a rolling-day average of the
    given size applied to the given columns."""
    rolling_avg_df = df.copy().reset_index(drop=True)
    # An N-day rolling average with N/2 days before and N/2 after requires N+1
    # days to include the day of as well.
    rolling_avg_df[columns] = (
        rolling_avg_df[columns].rolling(N + 1, center=True).mean())
    return rolling_avg_df

def smoothen_within(df, columns=[], by=None, N=6):
    """Returns a copy of the given dataframe with a rolling-day average of the
    given size applied to the given columns within each attribute class
    determined by the given "by" attribute."""
    attr_classes = set(df[by])
    rolling_dfs = []
    for attr_class in attr_classes:
        attr_class_df = df[df[by] == attr_class]
        rolling_dfs.append(smoothen(attr_class_df, columns, N=N))
    rolling_df = pd.concat(rolling_dfs)
    rolling_df = rolling_df.sort_values(by=['date_range_start', by])
    return rolling_df

In [ ]:
home_df = pd.read_csv(f'./exports/home_weekly.csv')
repair_dates(home_df)
home_df = home_df[home_df['date_range_start'].dt.year.isin([2019, 2020])]
home_df = home_df.rename(columns={'home_cbg': 'cbg'})
home_df.head()

,placekey,date_range_start,cbg,visitor_count,estimated_visitor_count,pct_visitor_count,pct_estimated_visitor_count,cdi
95,22t-222@627-s7m-rtv,2019-01-07 00:00:00-05:00,360050001000,4,0.000000,0.333333,0.000000,0.000000
96,22t-222@627-s7m-rtv,2019-01-07 00:00:00-05:00,360470206001,4,9.790000,0.021739,0.021739,2.772665
97,22t-222@627-s7m-rtv,2019-01-07 00:00:00-05:00,360470435003,4,8.605042,0.016129,0.016129,2.437068
98,22t-222@627-s7m-rtv,2019-01-07 00:00:00-05:00,360470437001,4,11.749526,0.024390,0.024390,3.327630
99,22t-222@627-s7m-rtv,2019-01-07 00:00:00-05:00,360470573002,5,1.068101,0.005160,0.005160,0.302501


In [ ]:
poi_df = pd.read_csv(f'./exports/poi_health_recategorized.csv')
poi_ffr_df = poi_df
poi_ffr_df['is_fast_food'] = (poi_ffr_df['category'] == 'Fast-Food Restaurants')
poi_ffr_df = poi_ffr_df[['placekey', 'is_fast_food']]
peek(poi_ffr_df)

,placekey,is_fast_food
0,226-222@627-s4n-pqf,False
1,225-225@627-s99-9xq,False
2,225-225@627-vsw-7nq,False


36467


In [ ]:
home_df = home_df.merge(poi_ffr_df, on=['placekey'], how='inner')
peek(home_df)

In [ ]:
home_df['estimated_ffr_visitor_count'] = 0

In [ ]:
home_df.loc[home_df['is_fast_food'],
            'estimated_ffr_visitor_count'] = home_df['estimated_visitor_count']
peek(home_df)

In [ ]:
def create_diff_df(metric_df, diff_columns=[], keep_columns=[]):
    metric_week_df = metric_df
    metric_week_df['year'] = metric_week_df['date_range_start'].dt.year
    metric_week_df['week'] = metric_week_df['date_range_start'].dt.week

    # Dates are missing from December 2020!

    metric_2020_df = metric_week_df[metric_week_df['year'] == 2020]
    metric_2020_df = metric_2020_df[metric_2020_df['week'] >= 2]
    metric_2020_df = metric_2020_df[metric_2020_df['week'] <= 52].reset_index(
        drop=True)
    metric_2019_df = metric_week_df[metric_week_df['year'] == 2019]
    metric_2019_df = metric_2019_df[metric_2019_df['week'] >= 2]
    # Remove dates that can't be compared.
    metric_2019_df = metric_2019_df[~metric_2019_df['week'].isin(set([50, 51]))]
    metric_2019_df = metric_2019_df[metric_2019_df['week'] <= 52].reset_index(
        drop=True)

    metric_diff_df = pd.DataFrame()
    for keep_column in (keep_columns + ['week', 'date_range_start']):
        metric_diff_df[keep_column] = metric_2020_df[keep_column]
    metric_diff_df[diff_columns] = (metric_2020_df[diff_columns] -
                                    metric_2019_df[diff_columns])
    metric_diff_df = metric_diff_df.dropna()

    peek(metric_diff_df.head())

    return metric_diff_df

In [ ]:
trips_df = home_df.groupby(by=['date_range_start']).agg({
    'estimated_visitor_count': 'sum',
    'estimated_ffr_visitor_count': 'sum',
}).reset_index()
trips_df['pct_ffr'] = trips_df['estimated_ffr_visitor_count'] / trips_df[
    'estimated_visitor_count']
peek(trips_df)

In [ ]:
trips_diff_df = create_diff_df(trips_df, diff_columns=['pct_ffr'])
trips_diff_df = smoothen(trips_diff_df, columns=['pct_ffr'])

In [ ]:
fig, ax = plt.subplots(figsize=(16, 6))
ax.plot(trips_diff_df['date_range_start'],
        trips_diff_df['pct_ffr'],
        color='C0')
ax.set_title('Diff in % of Trips to FFRs (52 Weeks)')
plt.tight_layout()
plt.show()

In [ ]:
cbg_df = pd.read_csv('./exports/proximity_clusters.csv')
cbg_df = cbg_df[['cbg', 'cluster', 'population']]
peek(cbg_df)

In [ ]:
merge_df = home_df.merge(cbg_df, how='inner', on=['cbg'])
peek(merge_df)

In [ ]:
cluster_df = merge_df.groupby(by=['date_range_start', 'cluster']).agg({
    'estimated_visitor_count': 'sum',
    'estimated_ffr_visitor_count': 'sum',
}).reset_index()
cluster_df = cluster_df.sort_values(by=['date_range_start', 'cluster'])
cluster_df['pct_ffr'] = cluster_df['estimated_ffr_visitor_count'] / cluster_df[
    'estimated_visitor_count']
cluster_df = smoothen_within(cluster_df, columns=['pct_ffr'], by='cluster')
peek(cluster_df)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 8))
for key, group in cluster_df.groupby(by=['cluster']):
    ax.plot(group['date_range_start'], group['pct_ffr'], label=key)
ax.set_title('% of Trips to FFRs')
ax.axvline(datetime.datetime(2020, 3, 20), linestyle='--')
ax.legend()
plt.show()

In [ ]:
proximity_df = pd.read_csv('./exports/proximity_count_index_sampled.csv')
peek(proximity_df)

In [ ]:
median_index = np.median(proximity_df['category_0_proximity_index'])
proximity_df['near_ffr'] = proximity_df['category_0_proximity_index'] > median_index
median_index = np.median(proximity_df['category_1_proximity_index'])
proximity_df['near_ssfs'] = proximity_df['category_1_proximity_index'] > median_index
proximity_df['near_ffr_only'] = (~proximity_df['near_ssfs'] & proximity_df['near_ffr'])
proximity_df['near_ssfs_only'] = (proximity_df['near_ssfs'] & ~proximity_df['near_ffr'])
peek(proximity_df)

In [ ]:
merge_proximity_df = merge_df.merge(proximity_df, how='inner', on=['cbg'])
peek(merge_proximity_df)

In [ ]:
near_ffr_df = merge_proximity_df.groupby(
    by=['date_range_start', 'near_ffr']).agg({
        'estimated_visitor_count': 'sum',
        'estimated_ffr_visitor_count': 'sum',
    }).reset_index()
near_ffr_df['pct_ffr'] = near_ffr_df[
    'estimated_ffr_visitor_count'] / near_ffr_df['estimated_visitor_count']
near_ffr_diff_df = create_diff_df(near_ffr_df,
                                  diff_columns=['pct_ffr'],
                                  keep_columns=['near_ffr'])
near_ffr_df = smoothen_within(near_ffr_df, columns=['pct_ffr'], by='near_ffr')
near_ffr_diff_df = smoothen_within(near_ffr_diff_df,
                                   columns=['pct_ffr'],
                                   by='near_ffr')

In [ ]:
near_ffr_only_df = merge_proximity_df.groupby(
    by=['date_range_start', 'near_ffr_only']).agg({
        'estimated_visitor_count': 'sum',
        'estimated_ffr_visitor_count': 'sum',
    }).reset_index()
near_ffr_only_df['pct_ffr'] = near_ffr_only_df[
    'estimated_ffr_visitor_count'] / near_ffr_only_df['estimated_visitor_count']
near_ffr_only_diff_df = create_diff_df(near_ffr_only_df,
                                       diff_columns=['pct_ffr'],
                                       keep_columns=['near_ffr_only'])
near_ffr_only_df = smoothen_within(near_ffr_only_df,
                                   columns=['pct_ffr'],
                                   by='near_ffr_only')
near_ffr_only_diff_df = smoothen_within(near_ffr_only_diff_df,
                                        columns=['pct_ffr'],
                                        by='near_ffr_only')

In [ ]:
near_ssfs_df = merge_proximity_df.groupby(
    by=['date_range_start', 'near_ssfs']).agg({
        'estimated_visitor_count': 'sum',
        'estimated_ffr_visitor_count': 'sum',
    }).reset_index()
near_ssfs_df['pct_ffr'] = near_ssfs_df[
    'estimated_ffr_visitor_count'] / near_ssfs_df['estimated_visitor_count']
near_ssfs_diff_df = create_diff_df(near_ssfs_df,
                                       diff_columns=['pct_ffr'],
                                       keep_columns=['near_ssfs'])
near_ssfs_df = smoothen_within(near_ssfs_df,
                                   columns=['pct_ffr'],
                                   by='near_ssfs')
near_ssfs_diff_df = smoothen_within(near_ssfs_diff_df,
                                        columns=['pct_ffr'],
                                        by='near_ssfs')

In [ ]:
near_ssfs_only_df = merge_proximity_df.groupby(
    by=['date_range_start', 'near_ssfs_only']).agg({
        'estimated_visitor_count': 'sum',
        'estimated_ffr_visitor_count': 'sum',
    }).reset_index()
near_ssfs_only_df['pct_ffr'] = near_ssfs_only_df[
    'estimated_ffr_visitor_count'] / near_ssfs_only_df['estimated_visitor_count']
near_ssfs_only_diff_df = create_diff_df(near_ssfs_only_df,
                                       diff_columns=['pct_ffr'],
                                       keep_columns=['near_ssfs_only'])
near_ssfs_only_df = smoothen_within(near_ssfs_only_df,
                                   columns=['pct_ffr'],
                                   by='near_ssfs_only')
near_ssfs_only_diff_df = smoothen_within(near_ssfs_only_diff_df,
                                        columns=['pct_ffr'],
                                        by='near_ssfs_only')

In [ ]:
fig, ax = plt.subplots(4, 2, figsize=(20, 16))

for key, group in near_ffr_df.groupby(by=['near_ffr']):
    ax[0, 0].plot(group['date_range_start'], group['pct_ffr'], label=key)
ax[0, 0].set_title('% of Trips to FFRs (Top 50% FFR Proximity)')

for key, group in near_ffr_diff_df.groupby(by=['near_ffr']):
    ax[0, 1].plot(group['date_range_start'], group['pct_ffr'], label=key)
ax[0, 1].set_title('Diff % of Trips to FFRs (Top 50% FFR Proximity)')

for key, group in near_ssfs_df.groupby(by=['near_ssfs']):
    ax[1, 0].plot(group['date_range_start'], group['pct_ffr'], label=key)
ax[1, 0].set_title('% of Trips to FFRs (Top 50% SSFS Proximity)')

for key, group in near_ssfs_diff_df.groupby(by=['near_ssfs']):
    ax[1, 1].plot(group['date_range_start'], group['pct_ffr'], label=key)
ax[1, 1].set_title('Diff % of Trips to FFRs (Top 50% SSFS Proximity)')

for key, group in near_ffr_only_df.groupby(by=['near_ffr_only']):
    ax[2, 0].plot(group['date_range_start'], group['pct_ffr'], label=key)
ax[2, 0].set_title('% of Trips to FFRs (Top 50% FFR Bottom 50% SSFS Proximity)')

for key, group in near_ffr_only_diff_df.groupby(by=['near_ffr_only']):
    ax[2, 1].plot(group['date_range_start'], group['pct_ffr'], label=key)
ax[2, 1].set_title(
    'Diff % of Trips to FFRs (Top 50% FFR Bottom 50% SSFS Proximity)')

for key, group in near_ssfs_only_df.groupby(by=['near_ssfs_only']):
    ax[3, 0].plot(group['date_range_start'], group['pct_ffr'], label=key)
ax[3, 0].set_title('% of Trips to FFRs (Bottom 50% FFR Top 50% SSFS Proximity)')

for key, group in near_ssfs_only_diff_df.groupby(by=['near_ssfs_only']):
    ax[3, 1].plot(group['date_range_start'], group['pct_ffr'], label=key)
ax[3, 1].set_title(
    'Diff % of Trips to FFRs (Bottom 50% FFR Top 50% SSFS Proximity)')

ax[0, 0].legend()
ax[0, 1].legend()
ax[1, 0].legend()
ax[1, 1].legend()
ax[2, 0].legend()
ax[2, 0].legend()
ax[3, 1].legend()
ax[3, 1].legend()

ax[0, 0].axvline(datetime.datetime(2020, 3, 20), linestyle='--')
ax[0, 1].axvline(datetime.datetime(2020, 3, 20), linestyle='--')
ax[1, 0].axvline(datetime.datetime(2020, 3, 20), linestyle='--')
ax[1, 1].axvline(datetime.datetime(2020, 3, 20), linestyle='--')
ax[2, 0].axvline(datetime.datetime(2020, 3, 20), linestyle='--')
ax[2, 1].axvline(datetime.datetime(2020, 3, 20), linestyle='--')
ax[3, 0].axvline(datetime.datetime(2020, 3, 20), linestyle='--')
ax[3, 1].axvline(datetime.datetime(2020, 3, 20), linestyle='--')

plt.show()

In [ ]:
cbg_gdf = gpd.read_file('./data/nyc_cbgs.geojson')
cbg_gdf = cbg_gdf.rename(columns={
    'CensusBlockGroup': 'cbg',
})
cbg_gdf['cbg'] = cbg_gdf['cbg'].astype(int)
cbg_gdf = cbg_gdf[['cbg', 'geometry']]

cbg_gdf = cbg_gdf.merge(
    proximity_df[['cbg', 'near_ffr', 'near_ssfs', 'near_ffr_only', 'near_ssfs_only']],
    on=['cbg'],
    how='inner')

cbg_gdf['color_ffr'] = 'C' + cbg_gdf['near_ffr'].astype(int).astype(str)
cbg_gdf['color_ssfs'] = 'C' + cbg_gdf['near_ssfs'].astype(int).astype(str)
cbg_gdf['color_ffr_only'] = 'C' + cbg_gdf['near_ffr_only'].astype(int).astype(str)
cbg_gdf['color_ssfs_only'] = 'C' + cbg_gdf['near_ssfs_only'].astype(int).astype(str)

cbg_df = pd.read_csv('./data/cbg_attr_and_cluster_1021.csv')
cbg_df = cbg_df.rename(columns={
    'census_block_group': 'cbg',
    'Median Household Income': 'income',
    'Total Population': 'population',
})
cbg_df = cbg_df[['cbg', 'population', 'income']]

cbg_gdf = cbg_gdf.merge(cbg_df, on=['cbg'], how='inner')
cbg_gdf['area'] = cbg_gdf['geometry'].area * 1000 * 1000

# XXX: Areas are already population-dependent.
cbg_gdf['population_density'] = cbg_gdf['population'] / cbg_gdf['area']
peek(cbg_gdf)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16, 8))
cbg_gdf.plot(ax=ax[0], color=cbg_gdf['color_ffr'])
cbg_gdf.plot(ax=ax[1], color=cbg_gdf['color_ssfs'])
ax[0].set_title('Top 50% FFR Proximity')
ax[1].set_title('Top 50% SSFS Proximity')
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16, 8))
cbg_gdf.plot(ax=ax[0], color=cbg_gdf['color_ffr_only'])
cbg_gdf.plot(ax=ax[1], color=cbg_gdf['color_ssfs_only'])
ax[0].set_title('Top 50% FFR Bottom 50% SSFS Proximity')
ax[1].set_title('Top 50% SSFS Bottom 50% FFR Proximity')
plt.tight_layout()
plt.show()

In [ ]:
median_index = np.median(cbg_gdf['population_density'])
cbg_gdf['is_dense'] = cbg_gdf['population_density'] > median_index
cbg_gdf['color_is_dense'] = 'C' + cbg_gdf['is_dense'].astype(int).astype(str)

fig, ax = plt.subplots(1, 1, figsize=(8, 8))
cbg_gdf.plot(ax=ax, color=cbg_gdf['color_is_dense'])
ax.set_title('Top 50% Population Density')
plt.tight_layout()
plt.show()

In [ ]:
merge_density_df = merge_df.merge(cbg_gdf[['cbg', 'is_dense']], how='inner', on=['cbg'])
dense_df = merge_density_df.groupby(
    by=['date_range_start', 'is_dense']).agg({
        'estimated_visitor_count': 'sum',
        'estimated_ffr_visitor_count': 'sum',
    }).reset_index()
dense_df['pct_ffr'] = dense_df[
    'estimated_ffr_visitor_count'] / dense_df['estimated_visitor_count']
dense_diff_df = create_diff_df(dense_df,
                                  diff_columns=['pct_ffr'],
                                  keep_columns=['is_dense'])
dense_df = smoothen_within(dense_df, columns=['pct_ffr'], by='is_dense')
dense_diff_df = smoothen_within(dense_diff_df,
                                   columns=['pct_ffr'],
                                   by='is_dense')

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 10))

for key, group in dense_df.groupby(by=['is_dense']):
    ax[0].plot(group['date_range_start'], group['pct_ffr'], label=key)
ax[0].set_title('% of Trips to FFRs (Top 50% Population Density)')

for key, group in dense_diff_df.groupby(by=['is_dense']):
    ax[1].plot(group['date_range_start'], group['pct_ffr'], label=key)
ax[1].set_title('Diff % of Trips to FFRs (Top 50% Population Density)')

ax[0].legend()
ax[1].legend()

ax[0].axvline(datetime.datetime(2020, 3, 20), linestyle='--')
ax[1].axvline(datetime.datetime(2020, 3, 20), linestyle='--')

plt.show()

In [ ]:
# XXX: Use NYC median income not median of CBG median income.
#median_index = np.median(cbg_gdf['income'].dropna())
median_index = 63998
cbg_gdf['is_high_income'] = cbg_gdf['income'] > median_index
cbg_gdf['color_is_high_income'] = 'C' + cbg_gdf['is_high_income'].astype(int).astype(str)

fig, ax = plt.subplots(1, 1, figsize=(8, 8))
cbg_gdf.plot(ax=ax, color=cbg_gdf['color_is_high_income'])
ax.set_title('Top 50% Median Household Income')
plt.tight_layout()
plt.show()

In [ ]:
merge_income_df = merge_df.merge(cbg_gdf[['cbg', 'is_high_income']], how='inner', on=['cbg'])
income_df = merge_income_df.groupby(
    by=['date_range_start', 'is_high_income']).agg({
        'estimated_visitor_count': 'sum',
        'estimated_ffr_visitor_count': 'sum',
    }).reset_index()
income_df['pct_ffr'] = income_df[
    'estimated_ffr_visitor_count'] / income_df['estimated_visitor_count']
income_diff_df = create_diff_df(income_df,
                                  diff_columns=['pct_ffr'],
                                  keep_columns=['is_high_income'])
income_df = smoothen_within(income_df, columns=['pct_ffr'], by='is_high_income')
income_diff_df = smoothen_within(income_diff_df,
                                   columns=['pct_ffr'],
                                   by='is_high_income')

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 10))

for key, group in income_df.groupby(by=['is_high_income']):
    ax[0].plot(group['date_range_start'], group['pct_ffr'], label=key)
ax[0].set_title('% of Trips to FFRs (Top 50% Income)')

for key, group in income_diff_df.groupby(by=['is_high_income']):
    ax[1].plot(group['date_range_start'], group['pct_ffr'], label=key)
ax[1].set_title('Diff % of Trips to FFRs (Top 50% Income)')

ax[0].legend()
ax[1].legend()

ax[0].axvline(datetime.datetime(2020, 3, 20), linestyle='--')
ax[1].axvline(datetime.datetime(2020, 3, 20), linestyle='--')

plt.show()

In [ ]:
cbg_gdf['is_low_density_high_income'] = cbg_gdf['is_dense'] & cbg_gdf['is_high_income']
merge_income_density_df = merge_df.merge(cbg_gdf[['cbg', 'is_low_density_high_income']], how='inner', on=['cbg'])
income_density_df = merge_income_density_df.groupby(
    by=['date_range_start', 'is_low_density_high_income']).agg({
        'estimated_visitor_count': 'sum',
        'estimated_ffr_visitor_count': 'sum',
    }).reset_index()
income_density_df['pct_ffr'] = income_density_df[
    'estimated_ffr_visitor_count'] / income_density_df['estimated_visitor_count']
income_density_diff_df = create_diff_df(income_density_df,
                                  diff_columns=['pct_ffr'],
                                  keep_columns=['is_low_density_high_income'])
income_density_df = smoothen_within(income_density_df, columns=['pct_ffr'], by='is_low_density_high_income')
income_density_diff_df = smoothen_within(income_density_diff_df,
                                   columns=['pct_ffr'],
                                   by='is_low_density_high_income')

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 10))

for key, group in income_density_df.groupby(by=['is_low_density_high_income']):
    ax[0].plot(group['date_range_start'], group['pct_ffr'], label=key)
ax[0].set_title('% of Trips to FFRs (Bottom 50% Density Top 50% Income)')

for key, group in income_density_diff_df.groupby(by=['is_low_density_high_income']):
    ax[1].plot(group['date_range_start'], group['pct_ffr'], label=key)
ax[1].set_title('Diff % of Trips to FFRs (Bottom 50% Density Top 50% Income)')

ax[0].legend()
ax[1].legend()

ax[0].axvline(datetime.datetime(2020, 3, 20), linestyle='--')
ax[1].axvline(datetime.datetime(2020, 3, 20), linestyle='--')

plt.show()

In [ ]:
# Children
# Driving